In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
import pandas as pd

from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager

In [2]:
filter = ['이수영역', '년도학기', '교과목명', '학점', '성적', '구이수']

# 웹 드라이버 경로 설정
driver = webdriver.Chrome(service=ChromeService(ChromeDriverManager().install())) # 크롬 버전에 따른 chromedriver.exe를 설치해야 하므로 해당 버전에 맞는 chromedriver 설치
main_window_handle = driver.current_window_handle

# 로그인 페이지로 이동
login_url = 'https://www.daejin.ac.kr/subLogin/daejin/view.do?layout=unknown'
driver.get(login_url)

username = "20191516"
password = "cho6090^)()"

In [4]:
# 로그인 폼 채우기
username_input = driver.find_element(By.ID, 'userId')
password_input = driver.find_element(By.ID, 'userPwd')

username_input.send_keys(username)
password_input.send_keys(password)

# 로그인 버튼 클릭
login_button = driver.find_element(By.CLASS_NAME, '_loginSubmit')
login_button.click()

time.sleep(1)

if 'message.do' in driver.current_url:
    driver.find_element(By.XPATH, '/html/body/div[1]/form/div/div/div[2]/div/span/input').click()

url = 'https://dreams2.daejin.ac.kr/sugang/LinkPortal.jsp'  # 이동하려는 URL로 수정하세요
driver.get(url)
time.sleep(1)

In [5]:
new_window_handle = None
for handle in driver.window_handles:
    if handle != main_window_handle:
        new_window_handle = handle
        break

if new_window_handle:
    driver.switch_to.window(new_window_handle)
    driver.close()  # 새로운 창 닫기
    driver.switch_to.window(main_window_handle)

#테스트 시작
driver.switch_to.frame('BBF')
driver.switch_to.frame('LF')
driver.find_element(By.ID, 'MFX8').click()

element = driver.find_element(By.ID, 'MFX9')
element.find_element(By.TAG_NAME, 'a').click()

In [6]:
#이수구분별 성적조회 및 출력클릭
driver.switch_to.parent_frame()
driver.switch_to.frame('RF')
driver.find_element(By.XPATH, '/html/body/table/tbody/tr/td/table[2]/tbody/tr[3]/td[1]/a').click()
time.sleep(1)
driver.find_element(By.CLASS_NAME, "crownix-absolute-100").click() # 화면 세로 맞춤
driver.find_element(By.ID, "crownix-toolbar-height").click()

# m2soft-crownix-text ID를 갖는 DIV 요소 찾기
time.sleep(1)
div_element = driver.find_element(By.ID, 'm2soft-crownix-text')

# DIV 요소의 모든 하위 요소들 가져오기
sub_elements = div_element.find_elements(By.XPATH, ".//*")

In [16]:
Course = {}
Course['교필'] = {'학점' : 0, '교과목명' : []}
Course['교선1'] = {'학점' : 0, '교과목명' : []}
Course['교선2'] = {'학점' : 0, '교과목명' : []}
Course['교선3'] = {'학점' : 0, '교과목명' : []}
Course['교선4'] = {'학점' : 0, '교과목명' : []}
Course['교선5'] = {'학점' : 0, '교과목명' : []}
Course['교선6'] = {'학점' : 0, '교과목명' : []}
Course['교선7'] = {'학점' : 0, '교과목명' : []}
Course['교선8'] = {'학점' : 0, '교과목명' : []}
Course['교선8'] = {'학점' : 0, '교과목명' : []}
Course['전필'] = {'학점' : 0, '교과목명' : []}
Course['전선'] = {'학점' : 0, '교과목명' : []}
Course['일선'] = {'학점' : 0, '교과목명' : []}
Course['부전1'] = {'학점' : 0, '교과목명' : []}
Course['복전1'] = {'학점' : 0, '교과목명' : []}

In [17]:
i = 19

while i < len(sub_elements):
    text = sub_elements[i].text
    if "일선 취득학점 계 : " in text:
        break
    elif ' 계 : ' in text:
        i += 2
    elif text not in filter:
        if text == '부전1' or text != '':
            skip = 7 if text == '부전1' else 6
            
            if len(text) > 2:
                text = text[0:3]
            Course[text]['학점'] += int(sub_elements[i+4].text)
            Course[text]['교과목명'].append(sub_elements[i+3].text)
            i += skip
        else:
            i += 1
    else:
        i += 1

19
25
31
37
43
51
57
63
69
75
81
87
93
99
105
111
117
125
131
137
143
151
157
163
169
175
181
187
193
199
205
211
217
223
229
235
241
249
256
263
270
277
284
293
299
305
311


In [20]:
Course

{'교필': {'학점': 8,
  '교과목명': ['비전설계및상담A', '사고와표현', '비전설계및상담A', '영어읽기와토론', '대순사상의이해']},
 '교선1': {'학점': 0, '교과목명': []},
 '교선2': {'학점': 2, '교과목명': ['음식과식량의문화사']},
 '교선3': {'학점': 2, '교과목명': ['스포츠마케팅인사이트']},
 '교선4': {'학점': 4, '교과목명': ['알기쉬운빅데이터와통계학의이해', '정보사회와컴퓨터']},
 '교선5': {'학점': 2, '교과목명': ['서양음악의역사와감상']},
 '교선6': {'학점': 0, '교과목명': []},
 '교선7': {'학점': 5, '교과목명': ['토익문법', '명사와함께하는나의미래설계']},
 '교선8': {'학점': 14,
  '교과목명': ['빅데이터분석의이해', '기초통계학', '제4차산업혁명과자기경영', 'OFFICE실무', '성격심리학']},
 '전필': {'학점': 12, '교과목명': ['컴퓨터시스템구조', '운영체제론', '컴퓨터알고리즘', '데이터베이스론']},
 '전선': {'학점': 48,
  '교과목명': ['컴퓨터프로그래밍기초',
   '콘텐츠기술의이해',
   '창의적공학프로그래밍입문',
   '객체지향프로그래밍',
   '컴퓨터시스템기초실습',
   '전산수학',
   '논리회로및실습',
   '자료구조',
   '시스템프로그래밍',
   '파일처리론',
   '자바프로그래밍',
   '데이터베이스설계',
   '고급시스템프로그래밍',
   '컴퓨터네트워크및보안',
   '웹프로그래밍',
   '프론트엔드웹디자인']},
 '일선': {'학점': 12, '교과목명': ['기초프로그래밍', '빅데이터분석심화', '정형DB실무', 'AI개론']},
 '부전1': {'학점': 18,
  '교과목명': ['원가.관리회계', '회계원리', '기업재무', '서비스운영관리', '인적자원관리', '조직행동론']},
 '복전1': {'학점': 0, '교과목